In [1]:
import os
import sys

from typing import Tuple, List, Optional
from qiskit import QuantumCircuit

script_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(script_dir)

from scripts.agents.RL_agent import RLMergeOptimizer


GRIDSYNTH = "gridsynth"
SOLOVAY_KITAEV = "solovay_kitaev"

KAK = "kak"
BQSKit = "bqskit"


In [2]:
from qiskit.circuit.random import random_circuit

my_circuit = random_circuit(num_qubits=5, depth=12, max_operands=2, seed=42)
circuit_name = "my_random_circuit"
my_circuit.draw(fold=-1)

┌─────────────────────────┐                  ┌───┐      ┌────────────┐            ┌───┐                                                                         ┌───────────────────┐         ┌───┐      ┌────────────────────────────┐       ┌────────────┐                                                                                                                    
q_0: ┤0                        ├──────────────────┤ H ├──────┤ Rx(4.6795) ├────────────┤ H ├──────────────────────────────────────────────■───────────────────■──────┤ U2(4.1589,3.4999) ├─────────┤ I ├──────┤ U3(0.55072,0.74145,6.0438) ├───────┤ Rz(4.7659) ├────────────────────────────────────────────────────────────────────────■───────────────────────────────■───────────
     │                         │┌────────────┐    └─┬─┘      └───┬───┬────┘            ├───┴┐          ┌───┐   ┌──────┐                   │                 ┌─┴──┐   └───┬────────────┬──┘    ┌────┴───┴─────┐└────────────────────────────┘       └────────────┘                     ┌──────────────────────────┐                       │             ┌───────────┐     │           
q_1: ┤                         ├┤ Rz(2.3298) ├──────┼────────────┤ X ├─────────────────┤ √X ├──────────┤ Y ├───┤ √Xdg ├───────────────────┼─────────────────┤ Sx ├───────┤ Ry(1.8447) ├───────┤1             ├────────────────────────────────────────────────────────────────■───────┤ U3(2.8637,1.2715,1.9224) ├──────────────■────────┼─────────────┤ P(6.0228) ├─────┼───────────
     │  (XX-YY)(4.939,0.80496) │└────────────┘      │       ┌────┴───┴─────┐┌──────────┴────┴─────────┐├───┤   └──────┘                   │                 ├───┬┘       └───┬────┬───┘       │              │        ┌───────────┐                ┌────────────┐             │       └────────────┬─────────────┘┌─────┐┌──────┴──────┐ │          ┌──┴───────────┴───┐ │           
q_2: ┤                         ├─■──────────────────■───────┤1             ├┤ U(1.4257,4.2086,2.7467) ├┤ H ├──────────────────────────────┼─────────────────┤ H ├────────────┤ √X ├───────────┤  Ryy(2.9021) ├────────┤ P(1.0133) ├────────────────┤ U1(5.7088) ├─────────────┼────────────────────┼──────────────┤ Sdg ├┤ Ry(0.26147) ├─┼──────────┤ R(2.1794,3.7128) ├─┼───────────
     │                         │ │            ┌────────────┐│              │└──────────┬───┬──────────┘├───┤┌────────────┐                │                 └─┬─┘            └────┘           │              │        └───────────┘                └────────────┘       ┌─────┴──────┐             │              └─────┘└─────────────┘ │P(3.1038) └──────────────────┘ │P(0.14328) 
q_3: ┤1                        ├─┼────────────┤ Ry(2.2276) ├┤  Rxx(4.2917) ├───────────┤ Y ├───────────┤ Y ├┤ Rx(2.8835) ├────────────────┼───────────────────■───────────────────────────────┤0             ├──────────────────────────────────────────────────────────┤ Ry(4.3964) ├─────────────■─────────────────────────────────────■───────────────────────────────■───────────
     └─────────────────────────┘ │U1(2.8299)  ├────────────┤│              │       ┌───┴───┴────┐      └───┘└────────────┘┌───────────────┴────────────────┐┌───┐ ┌──────────────────────────┐└┬────────────┬┘           ┌──────┐           ┌──────────────────────────┐└────────────┘                                                                                               
q_4: ────────────────────────────■────────────┤ Ry(4.7632) ├┤0             ├───────┤ Rx(5.2319) ├─────────────────────────┤ U(4.9444,4.1774,4.4307,4.9055) ├┤ Y ├─┤ U3(1.4699,0.36633,1.768) ├─┤ Rz(3.1482) ├────────────┤ √Xdg ├───────────┤ U3(3.6393,1.1107,5.3823) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                              └────────────┘└──────────────┘       └────────────┘                         └────────────────────────────────┘└───┘ └──────────────────────────┘ └────────────┘            └──────┘           └──────────────────────────┘

In [3]:
from scripts.agents.base_agent import CliffordTSynthesizer

synthesis_config = {
    "EPS": 0.01,
    "DEPTH": 2,
    "MODE1": GRIDSYNTH,
    "MODE2": KAK
}

synthesizer = CliffordTSynthesizer(mode1=synthesis_config["MODE1"],
                                   mode2=synthesis_config["MODE2"],
                                   epsilon=synthesis_config["EPS"],
                                   recursion_degree=synthesis_config["DEPTH"])
synthesized_circ, t_counts = synthesizer(my_circuit)
print(f"Initial T-count: {sum(t_counts)}")

Initial T-count: 614


In [4]:
model_config = {
    "AGENT_CODE": "PPO",
    "TOTAL_TIMESTEPS": 1_000_000, #10_000_000
    "INITIAL_STEPS": 2048, #2048
    "BATCH_SIZE": 1024, #1024
    "ENTROPY_COEFFICIENT": 0.05,
    "EVAL_BEST_OF": 25,
    "ONLY_INTERACTIONS": True,
}

RL_agent = RLMergeOptimizer(
        synthesis_config=synthesis_config,
        target_circuit_and_name=(my_circuit, circuit_name),
        model_config=model_config,
        KAK_SYNTHESIS_CACHE=None,
        U3_SYNTHESIS_CACHE=None)


rl_sequence, rl_tcount = RL_agent.optimize()
RL_agent.report()


Target circuit name: my_random_circuit
Number of qubits in target circuit: 5
Preprocessing done. Circuit cleaned.
First synthesis completed in 1.82 seconds.
========== Initial Circuit Statistics ==========
Number of qubits:       5
Number of gates:        43
Initial T-count:        614
Time taken:             1.82 seconds
Possible merges:        8
Merge strategy:         8 / 10 qubit pairs selected (interacting only)


Used pairs: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (2, 3), (2, 4)]
========== Initial Circuit Statistics ==========
Number of qubits:       5
Number of gates:        43
Initial T-count:        614.0
Time taken:             0.12 seconds
Possible merges:        8 / 10 qubit pairs selected
Used pairs: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (2, 3), (2, 4)]
========== Initial Circuit Statistics ==========
Number of qubits:       5
Number of gates:        43
Initial T-count:        614.0
Time taken:             0.10 seconds
Possible merges:        8 / 10 qu

Output()

/media/HDD/danlb/projects_4TB/rl_merging/venv/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:6
7: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting 
modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first
with ``Monitor`` wrapper.
  warnings.warn(

Eval num_timesteps=10000, episode_reward=100.00 +/- 0.00

Episode length: 2.00 +/- 0.00

...

Episode length: 3.00 +/- 0.00

Eval num_timesteps=160000, episode_reward=136.00 +/- 0.00

Episode length: 3.00 +/- 0.00

Stopping training because there was no new best model in the last 11 evaluations

Training completed.
[{'r': np.float64(40.0), 'l': 4}, {'r': np.float64(114.0), 'l': 3}, {'r': np.float64(44.0), 'l': 1}, {'r': np.float64(-24.0), 'l': 2}, {'r': np.float64(44.0), 'l': 1}, {'r': np.float64(-24.0), 'l': 3}, {'r': np.float64(-6.0), 'l': 2}, {'r': np.float64(90.0), 'l': 2}, {'r': np.float64(-12.0), 'l': 2}, {'r': np.float64(8.0), 'l': 3}, {'r': np.float64(72.0), 'l': 6}, {'r': np.float64(46.0), 'l': 3}, {'r': np.float64(64.0), 'l': 4}, {'r': np.float64(38.0), 'l': 2}, {'r': np.float64(112.0), 'l': 6}, {'r': np.float64(62.0), 'l': 5}, {'r': np.float64(2.0), 'l': 2}, {'r': np.float64(-24.0), 'l': 1}, {'r': np.float64(58.0), 'l': 3}, {'r': np.float64(-68.0), 'l': 5}, {'r': np.float64(4.0), 'l': 4}, {'r': np.float64(0.0), 'l': 1}, {'r': np.float64(66.0), 'l': 3}, {'r': np.float64(-24.0), 'l': 1}, {'r': np.float64(-48.0), 'l': 3}, {'r': np.float64(136.0), 'l': 3}, {'r': np.float64(94.0), 'l': 2}, {'r': np.float64(94.0), 'l': 3}, {'r': np.float64(-4.0), 'l': 6}, {'r': np.float64